In [1]:
%load_ext autoreload
%autoreload 2
# add . to module name
import sys
sys.path.append('../src/')

In [2]:
from package.logger import Timed, rlog, setup
from package import storage
setup("INFO")

In [3]:
import os
import psutil
import time

import geopandas as gpd

from package import strtime
from package.osm import osm
from package.mcr import mcr
from package.mcr.mcr import MCR
from package.mcr.data import MCRGeoData

In [4]:
city_id = "Koeln"
stops_path = "../data/cleaned/stops.csv"
osm_path = osm.get_osm_path_from_city_id(city_id)

with Timed.info("Reading stops"):
	other_stops_df = storage.read_gdf(stops_path)

if not os.path.exists(osm_path) and city_id:
	rlog.info("Downloading OSM data")
	osm.download_city(city_id, osm_path)
else:
	rlog.info("Using existing OSM data")

osm_reader = osm.new_osm_reader(osm_path)

with Timed.info("Getting OSM graph"):
	nodes, edges = osm.get_graph_for_city_cropped_to_stops(osm_reader, other_stops_df)

[16:07:07] INFO     Reading stops                                                                   ]8;id=195916;file:///tmp/ipykernel_51353/1792129641.py\1792129641.py]8;;\:]8;id=32578;file:///tmp/ipykernel_51353/1792129641.py#5\5]8;;\

           INFO     Reading stops done (0.18 seconds)                                               ]8;id=371575;file:///tmp/ipykernel_51353/1792129641.py\1792129641.py]8;;\:]8;id=696761;file:///tmp/ipykernel_51353/1792129641.py#5\5]8;;\

           INFO     Using existing OSM data                                                        ]8;id=658664;file:///tmp/ipykernel_51353/1792129641.py\1792129641.py]8;;\:]8;id=379220;file:///tmp/ipykernel_51353/1792129641.py#12\12]8;;\

           INFO     Getting OSM graph                                                              ]8;id=584187;file:///tmp/ipykernel_51353/1792129641.py\1792129641.py]8;;\:]8;id=383463;file:///tmp/ipykernel_51353/1792129641.py#16\16]8;;\

           INFO     Loading OSM network from cache                                                 ]8;id=198403;file:///tmp/ipykernel_51353/1792129641.py\1792129641.py]8;;\:]8;id=888119;file:///tmp/ipykernel_51353/1792129641.py#16\16]8;;\

[16:07:08] INFO     Getting OSM graph done (0.98 seconds)                                          ]8;id=333825;file:///tmp/ipykernel_51353/1792129641.py\1792129641.py]8;;\:]8;id=357739;file:///tmp/ipykernel_51353/1792129641.py#16\16]8;;\

In [5]:
nodes

,lon,lat,tags,timestamp,version,changeset,id,geometry
0,6.932525,50.937519,"{'TMC:cid_58:tabcd_1:Class': None, 'TMC:cid_58...",0,0,0,21063145,POINT (6.93252 50.93752)
1,6.932627,50.937519,None,0,0,0,7151289920,POINT (6.93263 50.93752)
2,6.932735,50.937519,"{'TMC:cid_58:tabcd_1:Class': None, 'TMC:cid_58...",0,0,0,10929975,POINT (6.93274 50.93752)
3,6.934627,50.937499,None,0,0,0,10853912,POINT (6.93463 50.93750)
6,6.938906,50.936414,None,0,0,0,367149,POINT (6.93891 50.93641)
...,...,...,...,...,...,...,...,...
1215735,6.932704,50.962141,None,0,0,0,11105976044,POINT (6.93270 50.96214)
1215736,6.932744,50.962156,"{'TMC:cid_58:tabcd_1:Class': None, 'TMC:cid_58...",0,0,0,11105976043,POINT (6.93274 50.96216)
1215737,6.931031,50.959016,None,0,0,0,11105943462,POINT (6.93103 50.95902)
1215738,6.930911,50.959047,None,0,0,0,11105943463,POINT (6.93091 50.95905)


In [6]:
from h3 import h3
import folium
from typing import List, Set


def get_h3_cells_for_nodes(nodes: List[dict], resolution: int) -> Set[str]:
    h3_cells: Set[str] = set()

    for node in nodes:
        lat, lon = node["lat"], node["lon"]
        h3_cell = h3.geo_to_h3(lat, lon, resolution)
        h3_cells.add(h3_cell)

    return h3_cells


def plot_h3_cells_on_folium(h3_cells: Set[str], folium_map: folium.Map) -> None:
    for h3_cell in h3_cells:
        geo_boundary = list(h3.h3_to_geo_boundary(h3_cell))
        # Close the loop by appending the first coordinates at the end of the list
        geo_boundary.append(geo_boundary[0])
        folium.Polygon(
            locations=geo_boundary,
            color="blue",
            weight=2.5,
            opacity=1,
            fill_color="blue",
            fill_opacity=0.2,
        ).add_to(folium_map)


In [7]:
h3_cells = get_h3_cells_for_nodes(nodes[["lat", "lon"]].to_dict("records"), 9)
m = folium.Map(location=[50.9333, 6.95], zoom_start=12)
plot_h3_cells_on_folium(h3_cells, m)
m

In [8]:
from h3 import h3
from typing import List, Set

def get_h3_cells_for_bbox(min_lat: float, min_lon: float, max_lat: float, max_lon: float, resolution: int) -> Set[str]:
    h3_cells: Set[str] = set()

    # Get the edge length in meters for the H3 resolution
    edge_length_m = h3.edge_length(resolution, unit='m')
    
    # Approximate step size in degrees, assuming 111,111 meters per degree
    # This is a rough approximation and works best near the equator. 
    # The closer to the poles you get, the less accurate this becomes.
    step_size = edge_length_m / 111111.0


    lat = min_lat
    while lat <= max_lat:
        lon = min_lon
        while lon <= max_lon:
            h3_cell = h3.geo_to_h3(lat, lon, resolution)
            h3_cells.add(h3_cell)
            lon += step_size
        lat += step_size

    return h3_cells

In [36]:
# Initialize Folium Map centered around Cologne, Germany
m = folium.Map(location=[50.9375, 6.9603], zoom_start=12)


# Get unique H3 cells covering the OSM nodes at a given resolution (e.g., 9)
resolution = 9

bbox_cologne_center = [50.92, 6.94, 50.96, 6.98]
h3_cells = get_h3_cells_for_bbox(*bbox_cologne_center, resolution=9)

# Plot H3 cells on the Folium map
plot_h3_cells_on_folium(h3_cells, m)

# draw bbox
folium.Rectangle(
	bounds=[[bbox_cologne_center[0], bbox_cologne_center[1]], [bbox_cologne_center[2], bbox_cologne_center[3]]],
	color='red',
	fill=False,
).add_to(m)

# Show the map
m

In [10]:
from scipy.spatial import cKDTree
import numpy as np
import pandas as pd
from h3 import h3
from typing import List, Dict, Union

def get_closest_osm_nodes_to_h3_cells(h3_cells: List[str], osm_nodes_df: pd.DataFrame) -> Dict[str, Union[None, Dict[str, float]]]:
    closest_nodes: Dict[str, Union[None, Dict[str, float]]] = {}

    # Prepare a numpy array from osm_nodes_df for fast k-d tree query
    osm_nodes_array = osm_nodes_df[['lat', 'lon']].to_numpy()
    kdtree = cKDTree(osm_nodes_array)
    
    for h3_cell in h3_cells:
        # Get the center of the H3 cell
        cell_center = h3.h3_to_geo(h3_cell)
        
        # Query the k-d tree for the closest point
        distance, index = kdtree.query(np.array([cell_center]))

        node = osm_nodes_df.iloc[index].iloc[0]
        closest_node = {
            'lat': node.lat,
            'lon': node.lon,
            'distance': distance[0] * 111111,  # Convert to meters approximately
            'osm_node_id': node.id,
        }

        closest_nodes[h3_cell] = closest_node

    return closest_nodes

In [37]:
# h3_cells = get_h3_cells_for_nodes(nodes[["lat", "lon"]].to_dict("records"), 9)
h3_cells = get_h3_cells_for_bbox(*bbox_cologne_center, resolution=9)

In [38]:
osm_node_per_hex = get_closest_osm_nodes_to_h3_cells(h3_cells, nodes)

In [39]:
stops = "../data/cleaned/stops.csv"
city_id = "Koeln"
structs = "../data/structs.pkl"

In [40]:
n_first = 32

In [41]:
start_time="08:00:00"
max_transfers=2

# for h3_cell, osm_info in list(osm_node_per_hex.items())[:n_first]:
# 	osm_node_id = osm_info["osm_node_id"]

# 	mcr_runner = MCR(mcr_geo_data, disable_paths=True)

# 	start_node_id=osm_node_id
# 	output=f"../data/mcr5/{city_id}/{h3_cell}.pkl"

# 	mcr_runner.run(
# 		start_node_id=osm_node_id,
# 		start_time=start_time,
# 		max_transfers=max_transfers,
# 		output_path=output,
# 	)

In [42]:
def get_available_memory() -> int:
    return psutil.virtual_memory().available

def pretty_bytes(b: float) -> str:
	for unit in ["B", "KiB", "MiB", "GiB", "TiB", "PiB"]:
		if b < 1024:
			return f"{b:.2f}{unit}"
		b /= 1024
	return f"{b:.2f}EiB"

def get_active_process_count(processes) -> int:
    return sum(p.is_alive() for p in processes)

pretty_bytes(get_available_memory())

'21.45GiB'

In [44]:
MIN_FREE_MEMORY = 5 * 1024 ** 3  # 5 GiB

In [45]:
from package.mcr.output import OutputFormat

In [46]:
from multiprocessing import Process


def run_mcr(
    h3_cell: str,
    osm_info: dict,
    mcr_geo_data,
    start_time: str,
    max_transfers: int,
    city_id: str,
) -> None:
    # Your existing code
    osm_node_id = osm_info["osm_node_id"]
    mcr_runner = MCR(mcr_geo_data, disable_paths=True, output_format=OutputFormat.DF_FEATHER)
    output = f"../data/mcr5/{city_id}/{h3_cell}.feather"

    mcr_runner.run(
        start_node_id=osm_node_id,
        start_time=start_time,
        max_transfers=max_transfers,
        output_path=output,
    )


max_concurrent_processes = 12

processes = []

for h3_cell, osm_info in list(osm_node_per_hex.items()):
# for h3_cell, osm_info in list(osm_node_per_hex.items())[:n_first]:

    def print_status():
        available_memory = pretty_bytes(get_available_memory())
        active_processes = get_active_process_count(processes)
        started_processes = len(processes)
        total_processes = len(osm_node_per_hex)

        print(
            f"Available memory: {available_memory} | active: {active_processes} | started: {started_processes}/{total_processes}        ",
            end="\r",
        )

    # Check available memory
    while (
        get_active_process_count(processes) >= max_concurrent_processes
        or get_available_memory() < MIN_FREE_MEMORY
    ):
        print_status()
        time.sleep(1)  # Sleep for 1 second before checking again

    p = Process(
        target=run_mcr,
        args=(h3_cell, osm_info, mcr_geo_data, start_time, max_transfers, city_id),
    )
    # rlog.info(
    #     f"Starting process for H3 cell {h3_cell}. ({len(processes) + 1}/{n_first})"
    # )
    p.start()
    processes.append(p)

# Wait for processes to complete
for p in processes:
    p.join()


KeyboardInterrupt: 

In [21]:
seconds_for_32 = 2 * 60 + 18.3
seconds_for_1 = seconds_for_32 / 32
print(len(osm_node_per_hex) * seconds_for_1 / 60, "minutes")

95.08125000000001 minutes


In [88]:
import overpy

api = overpy.Overpass()

# Define bounding box for Cologne: [south,west,north,east]
bounding_box = "(50.8700,6.8000,51.0500,7.0500)"

# Overpass QL query for supermarkets in the bounding box
overpass_query = f"""
[out:json];
(
    node["shop"="supermarket"]{bounding_box};
    way["shop"="supermarket"]{bounding_box};
    relation["shop"="supermarket"]{bounding_box};
);
out center;
"""

result = api.query(overpass_query)

In [104]:
markets = [
	{
		"lat": float(node.lat),
		"lon": float(node.lon),
		"name": node.tags.get("name", "Unknown"),
		# "tags": node.tags,
		"id": node.id,
	}
	for node in result.nodes
]
markets = pd.DataFrame(markets)
markets.head()

,lat,lon,name,id
0,50.940872,7.008140,Netto Marken-Discount,55441040
1,50.946321,6.921813,Kaufland,102991868
2,50.958592,6.949800,Netto Marken-Discount,232289350
3,50.925526,6.958147,Netto City,242515981
4,50.954776,6.916874,REWE,243924635


In [129]:
area_of_interest = nodes.geometry.unary_union.convex_hull
markets = gpd.GeoDataFrame(markets, geometry=gpd.points_from_xy(markets.lon, markets.lat))
# only keep markets within area of interest
markets = markets[markets.within(area_of_interest)]
markets

,lat,lon,name,id,nearest_osm_node_id,distance,geometry
0,50.940872,7.008140,Netto Marken-Discount,55441040,2725476496,6.737762,POINT (7.00814 50.94087)
1,50.946321,6.921813,Kaufland,102991868,1246265182,9.802642,POINT (6.92181 50.94632)
2,50.958592,6.949800,Netto Marken-Discount,232289350,1680834812,19.672917,POINT (6.94980 50.95859)
3,50.925526,6.958147,Netto City,242515981,3890976981,22.818948,POINT (6.95815 50.92553)
4,50.954776,6.916874,REWE,243924635,6908409244,30.201176,POINT (6.91687 50.95478)
...,...,...,...,...,...,...,...
220,50.911179,6.942079,REWE,9272366953,795845340,15.449580,POINT (6.94208 50.91118)
221,50.936718,6.932119,ALDI Süd,9383815374,9244012204,31.671549,POINT (6.93212 50.93672)
222,50.949962,6.912206,Netto Marken-Discount,9425176148,9425176155,25.912260,POINT (6.91221 50.94996)
223,50.937015,6.899031,Go2Market,9547686027,7021491522,34.779245,POINT (6.89903 50.93702)


In [136]:
def add_closest_osm_node(df: pd.DataFrame, osm_nodes_df: pd.DataFrame) -> Dict[str, Union[None, Dict[str, float]]]:
    osm_nodes_array = osm_nodes_df[['lat', 'lon']].to_numpy()
    kdtree = cKDTree(osm_nodes_array)


    to_query = df[['lat', 'lon']].to_numpy()
    distance, index = kdtree.query(to_query)

    distance = distance * 111111
    nearest_node_ids = osm_nodes_df.iloc[index].id.values

    df["nearest_osm_node_id"] = nearest_node_ids
    df["distance"] = distance

    if df["distance"].max() > 1000:
        print("WARNING: max distance > 1000m")

    return df

In [137]:
markets = add_closest_osm_node(markets, nodes)
markets

,lat,lon,name,id,nearest_osm_node_id,distance,geometry
0,50.940872,7.008140,Netto Marken-Discount,55441040,2725476496,6.737762,POINT (7.00814 50.94087)
1,50.946321,6.921813,Kaufland,102991868,1246265182,9.802642,POINT (6.92181 50.94632)
2,50.958592,6.949800,Netto Marken-Discount,232289350,1680834812,19.672917,POINT (6.94980 50.95859)
3,50.925526,6.958147,Netto City,242515981,3890976981,22.818948,POINT (6.95815 50.92553)
4,50.954776,6.916874,REWE,243924635,6908409244,30.201176,POINT (6.91687 50.95478)
...,...,...,...,...,...,...,...
220,50.911179,6.942079,REWE,9272366953,795845340,15.449580,POINT (6.94208 50.91118)
221,50.936718,6.932119,ALDI Süd,9383815374,9244012204,31.671549,POINT (6.93212 50.93672)
222,50.949962,6.912206,Netto Marken-Discount,9425176148,9425176155,25.912260,POINT (6.91221 50.94996)
223,50.937015,6.899031,Go2Market,9547686027,7021491522,34.779245,POINT (6.89903 50.93702)


In [161]:
# read all files in folder
folder = "/home/moritz/dev/uni/mcr-py/data/mcr5/Koeln/"
files = os.listdir(folder)
for i, file_path in enumerate(files):
	data = storage.read_any_dict(os.path.join(folder, file_path))
	bags_i = data["bags_i"]
	print(f"{i}/{len(files)}\r", end="")

KeyboardInterrupt: 

In [138]:
data = storage.read_any_dict("/home/moritz/dev/uni/mcr-py/data/mcr5/Koeln/891fa18b5abffff.pkl")
bags_i = data["bags_i"]

In [145]:
labels = pd.DataFrame(
    [
        (label.node_id, label.values[0], label.values[1], n_transfers)
        for n_transfers, bags in bags_i.items()
        for bag in bags.values()
        for label in bag
    ],
    columns=["osm_node_id", "time", "cost", "n_transfers"],
)

labels["human_readable_time"] = labels["time"].apply(strtime.seconds_to_str_time)

,osm_node_id,time,cost,n_transfers,human_readable_time
0,8358935994,34053,0,0,09:27:33
1,84497443,31205,0,0,08:40:05
2,279586235,33805,0,0,09:23:25
3,1769362535,33558,0,0,09:19:18
4,1776109449,31047,0,0,08:37:27
...,...,...,...,...,...
415548,286566116,29945,0,2,08:19:05
415549,8657971003,30198,0,2,08:23:18
415550,8065779681,32249,0,2,08:57:29
415551,8065779681,31726,1,2,08:48:46


In [153]:
memory_usage = labels.memory_usage(deep=True).sum()
in_mb = memory_usage / 1024 ** 2
print(f"Memory usage: {in_mb:.2f} MiB")

Memory usage: 38.44 MiB


In [32]:
for hex_id in list(osm_node_per_hex.keys())[:10]:
	pd.read_feather(f"../data/mcr5/Koeln/{hex_id}.pkl")


,osm_node_id,time,cost,n_transfers,human_readable_time
0,208180408,37062,0,0,10:17:42
1,2893101297,33732,0,0,09:22:12
2,2047282120,36487,0,0,10:08:07
3,247409759,34232,0,0,09:30:32
4,5841277889,37369,0,0,10:22:49
...,...,...,...,...,...
448208,10163938380,30408,0,2,08:26:48
448209,1456574636,30916,0,2,08:35:16
448210,3223281130,30844,0,2,08:34:04
448211,1962291023,31418,0,2,08:43:38
